### End to end deep learning project using Simple RNN

In [30]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [31]:
### Load the IMDB data
max_features = 10000 # Vocabulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Test data shape: {X_test.shape}, Test labels shape: {y_test.shape}')


Training data shape: (25000,), Training labels shape: (25000,)
Test data shape: (25000,), Test labels shape: (25000,)


In [32]:
## Inspect a sample review and its label
sample_review = X_train[0]
sample_label = y_train[0]

print(f'Sample review (as integers): {sample_review}')
print(f'Sample label: {sample_label}')

Sample review (as integers): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample label: 1


In [33]:
## Mapping of words index back to words
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [34]:
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in sample_review])
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [35]:
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [36]:
## Train Simple RNN model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len)) # Empbedding layer
model.add(SimpleRNN(128, activation='relu')) # Simple RNN layer
model.add(Dense(1, activation='sigmoid')) # Output layer

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
## Create an instance of early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=10, batch_size=32, 
          validation_split=0.2, 
          callbacks=[early_stopping])

Epoch 1/10


625/625 [==============================] - 78s 123ms/step - loss: 0.7338 - acc: 0.6169 - val_loss: 0.5974 - val_acc: 0.6656
Epoch 2/10
625/625 [==============================] - 66s 105ms/step - loss: 0.5030 - acc: 0.7618 - val_loss: 0.4922 - val_acc: 0.7630
Epoch 3/10
625/625 [==============================] - 67s 107ms/step - loss: nan - acc: 0.5472 - val_loss: nan - val_acc: 0.5062
Epoch 4/10
625/625 [==============================] - 67s 107ms/step - loss: nan - acc: 0.4985 - val_loss: nan - val_acc: 0.5062
Epoch 5/10
625/625 [==============================] - 59s 94ms/step - loss: nan - acc: 0.4985 - val_loss: nan - val_acc: 0.5062
Epoch 6/10
625/625 [==============================] - 65s 105ms/step - loss: nan - acc: 0.4985 - val_loss: nan - val_acc: 0.5062
Epoch 7/10
625/625 [==============================] - 64s 103ms/step - loss: nan - acc: 0.4985 - val_loss: nan - val_acc: 0.5062


In [38]:
## Save model file 
model.save('simple_rnn_model.h5')

c:\Users\hsingh\OneDrive - INVIDI Technologies Corp\machine learning\machine-learning\myenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
